# Simple Model Training with Darts Library

In [1]:
#work with darts library to create a time series forecast
import darts
import pandas as pd
import wapi
from darts.models.forecasting.rnn_model import RNNModel
from darts import TimeSeries



c:\Users\thoma\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsforecast\core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
def assemble_dataframe(time_start, time_end, config_file_path='../apiconfig.ini'):
    curve_names = [
        "pro ch spv mwh/h cet min15 a",
        "pro ch spv mwh/h cet min15 n",
        "pro it spv mwh/h cet min15 a",
        "pro fr spv mwh/h cet min15 a",
        "pro de spv mwh/h cet min15 a",
        "pro at spv mwh/h cet min15 a",
        "cap ch spv mw cet min15 a",
        "tt ch grimsel test °c cet h s",
        "tt ch adelboden test °c cet h s",
        "tt ch geneve test °c cet h s",
        "tt ch davos test °c cet h s",
        "tt ch engelberg test °c cet h s",
        "tt ch bern test °c cet h s",
        "tt ch lugano test °c cet h s",
        "tt ch zurich test °c cet h s",
        "tt ch samedan test °c cet h s",
        "cc ch con ec00 % cet min15 f",
        "cc ch grimsel test % cet h s",
        "cc ch adelboden test % cet h s",
        "cc ch geneve test % cet h s",
        "cc ch davos test % cet h s",
        "cc ch engelberg test % cet h s",
        "cc ch bern test % cet h s",
        "cc ch lugano test % cet h s",
        "cc ch zurich test % cet h s",
        "cc ch samedan test % cet h s",
    ]
    session = wapi.Session(config_file=config_file_path)
    ch_total_prod_curve = session.get_curve(name=curve_names[0])
    ts = ch_total_prod_curve.get_data(data_from= time_start, data_to=time_end, frequency='h', function ="AVERAGE")
    return ts.to_pandas()

In [19]:
data = assemble_dataframe('2022-02-01', '2022-03-03')


c:\Users\thoma\AppData\Local\Programs\Python\Python312\Lib\site-packages\wapi\util.py:135: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return res.asfreq(self._map_freq(self.frequency))


In [20]:
pd_dataframe = pd.DataFrame({'timestamp':data.index, 'value':data.values})
pd_dataframe['timestamp'] = pd.to_datetime(pd_dataframe.timestamp).dt.tz_localize(None)
series = TimeSeries.from_dataframe(pd_dataframe, "timestamp", "value")

#localize tz from 

In [23]:
#select the training and validation 20% is validation and last points
train, val = series.split_after(0.8)

#types = ["LSTM", "GRU", "RNN"] available models
choosen_model = "LSTM"
chunk_length = 10
hidden_dim = 256
n_epochs = 100

#rnn model 
model = RNNModel(model=choosen_model,input_chunk_length= chunk_length, hidden_dim = 256 ,n_rnn_layers=1, n_epochs=n_epochs, random_state=0) 

#fit the model


model.fit(train)
prediction = model.predict(len(val))


import matplotlib.pyplot as plt
series.plot(label='actual')
prediction.plot(label='forecast', lw=3)
plt.legend()

